In [1]:
import pandas as pd
import json
import numpy as np
import networkx as nx


with open('data.json', 'r', encoding='utf-8') as json_data:
    data = json.load(json_data)

normalized_data = []

for key, value in data.items():
    user_id = key
    info = value.get('info', [{}])[0]  
    groups = value.get('groups', [])
    if not groups:
        record = {**info, 'id': user_id}
        normalized_data.append(record)
        continue
    for group in groups:
        record = {**info, **group, 'id': user_id, 'group_id': group['id']}
        normalized_data.append(record)

df = pd.DataFrame(normalized_data)

In [2]:
df.head()

,id,first_name,last_name,can_access_closed,is_closed,deactivated,name,screen_name,type,photo_50,photo_100,photo_200,group_id
0,114668685,Alexey,Dozorov,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64172662,Alexander,Antonnikov,True,False,deleted,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,256740,Mikhail,Podgorny,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,118465737,Renat,Khakimov,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90084913,Anna,Solovskaya,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.drop(['photo_50', 'photo_100', 'photo_200', 'deactivated', 'can_access_closed', 'is_closed', 'type'], axis=1)

In [4]:
df

,id,first_name,last_name,name,screen_name,group_id
0,114668685,Alexey,Dozorov,NaN,NaN,NaN
1,64172662,Alexander,Antonnikov,NaN,NaN,NaN
2,256740,Mikhail,Podgorny,NaN,NaN,NaN
3,118465737,Renat,Khakimov,NaN,NaN,NaN
4,90084913,Anna,Solovskaya,NaN,NaN,NaN
...,...,...,...,...,...,...
365358,794082984,Marina,Manerova,Путь к себе. Эзотерика.,prosnis24,46558304.0
365359,794082984,Marina,Manerova,Новости | Саранск City | Republic of Mordovia,gorod_saransk,36809318.0
365360,794082984,Marina,Manerova,САРАНСК Онлайн,rmsar,60178575.0
365361,794082984,Marina,Manerova,10 канал,10tvrm,70336234.0


In [5]:
df['group_id'] = df['group_id'].astype(float).astype('Int64')

In [6]:
df.rename(columns={'name':'group_name', 'screen_name': 'group_screen_name', 'id': 'user_id', 'first_name': 'user_first_name', 'last_name': 'user_last_name'}, inplace=True)

In [7]:
df.dropna(how='any', inplace=True)

In [8]:
df

,user_id,user_first_name,user_last_name,group_name,group_screen_name,group_id
7,125402825,Artyom,Amosov,Sperry Ukraine,sperrytopsiderukraine,19603458
8,125402825,Artyom,Amosov,Dubstep set up,dubstepforum,22778924
9,125402825,Artyom,Amosov,КИНОМАНИЯ | Фильмы и сериалы,kino_mania,22798006
10,125402825,Artyom,Amosov,MDK,mudachyo,23148107
11,125402825,Artyom,Amosov,Artist Music,artistmusic,23162459
...,...,...,...,...,...,...
365358,794082984,Marina,Manerova,Путь к себе. Эзотерика.,prosnis24,46558304
365359,794082984,Marina,Manerova,Новости | Саранск City | Republic of Mordovia,gorod_saransk,36809318
365360,794082984,Marina,Manerova,САРАНСК Онлайн,rmsar,60178575
365361,794082984,Marina,Manerova,10 канал,10tvrm,70336234


In [9]:
G = nx.Graph()

In [10]:
edgelist = df[['user_id', 'group_id']]

In [11]:
edgelist

,user_id,group_id
7,125402825,19603458
8,125402825,22778924
9,125402825,22798006
10,125402825,23148107
11,125402825,23162459
...,...,...
365358,794082984,46558304
365359,794082984,36809318
365360,794082984,60178575
365361,794082984,70336234


In [12]:
G1=nx.from_pandas_edgelist(edgelist, 'user_id', 'group_id', create_using=nx.Graph())

In [13]:
def map_df(input_df):
    group_values_df = pd.DataFrame(input_df, columns=['group_id', 'weight'])
    merged_df = pd.merge(group_values_df, df, on='group_id', how='left')
    result = [(row['group_id'], row['weight'], row['group_name']) for idx, row in merged_df.iterrows()]
    result_df = pd.DataFrame(result, columns=['group_id', 'weight', 'group_name'])
    return result_df


# Центральность по посредничеству
*Проблема в том, что временная сложность алгоритма равняется O(VE), что гарантирует гигантское время прогона. Но и на случайную выборку тоже сложно надеяться, потому что у всех людей различные группы. Но что-то общее все-таки присутствует*

**В алгоритме:** `k=100` **- случайные узлы графа**

In [14]:

betweenness_centrality = sorted(list(nx.betweenness_centrality(G1, k=100).items()), key=lambda i: i[1], reverse=True)

In [15]:
betweenness_centrality[:10]

[(91412129, 0.1106345369716823),
 (39325472, 0.07449546097964109),
 (28905875, 0.05779594005144411),
 (45745333, 0.04542922407099784),
 (66678575, 0.03451416942727679),
 (29573241, 0.029374283574297007),
 (91050183, 0.02505661429829731),
 (73424524, 0.023415842397204567),
 (73247559, 0.022033875660798567),
 (192271135, 0.021049781055495403)]

# Центральность по посредничеству - результат

*Если пользователь состоит в нескольких группах, то эти группы считаются связанными. Группы с высокой центральностью по посредничеству - это те группы, через которые проходит большое количество кратчайших путей между другими группами в сети*

*Рассмотрим группу* `Подслушано Рузаевка (САРАНСК)` *- она имеет самый высокий вес (0.110635), что указывает на её важную роль в соединении других групп в сети. Это означает, что многие пользователи, состоящие в других группах, также являются членами этой группы*

*Эта информация может быть полезна для выявления ключевых групп, которые связывают пользователей*

In [16]:
map_df(betweenness_centrality).drop_duplicates()

,group_id,weight,group_name
0,91412129,0.110635,Подслушано Рузаевка (САРАНСК)
2555,39325472,0.074495,Типичный Балаково!
4459,28905875,0.057796,Рифмы и Панчи
5993,45745333,0.045429,4ch
7084,66678575,0.034514,"Овсянка, сэр!"
...,...,...,...
340986,198285669,0.000000,Нейрографика. Психолог. Дарья Петровичева
340987,217306195,0.000000,Служба Семьи в Белорецком районе
340988,198725806,0.000000,"Живые открытки, видео."
340989,213430879,0.000000,Жильё для отдыха Дивноморск


# Центральность по собственному вектору

*Сам граф состоит из пользователей и их связей по группам, а значит многие узлы связаны напрямую или косвенно через друзей друзей. Такая плотная структура приводит к тому, что собственный вектор центральности сложно стабилизировать за небольшое количество итераций.*

`max_iter=600`  **означает, что алгоритму дается до 600 итераций для нахождения собственного вектора**

In [17]:
# Центральность по собственному вектору
eigenvector_centrality = sorted(list(nx.eigenvector_centrality(G1, max_iter=600).items()), key=lambda i: i[1], reverse=True)

In [18]:
eigenvector_centrality[:10]

[(91412129, 0.32813481783193293),
 (39325472, 0.24956744452439694),
 (28905875, 0.18292544412694056),
 (49097575, 0.15616414880096943),
 (192271135, 0.14188270114078158),
 (42122605, 0.14036050901868294),
 (45745333, 0.13294286355149387),
 (66678575, 0.12311751079437394),
 (73424524, 0.11308846038326499),
 (89246551, 0.11195158813740405)]

# Центральность по собственному вектору - результат

*Если пользователь состоит в нескольких группах, то эти группы считаются связанными. Чем больше связей имеет группа с другими важными группами (то есть группами с высоким значением центральности), тем выше будет её собственное значение центральности.*

*Вес группы* `weight` *показывает относительную важность группы в сети, учитывая важность связанных с ней групп. Группы с более высоким весом считаются более влиятельными и центральными в сети.*

*Эта информация позволяет определить наиболее важные и влиятельные группы в сети, основываясь на связях между пользователями и их друзьями.* 

In [19]:
map_df(eigenvector_centrality).drop_duplicates()

,group_id,weight,group_name
0,91412129,0.328135,Подслушано Рузаевка (САРАНСК)
2555,39325472,0.249567,Типичный Балаково!
4459,28905875,0.182925,Рифмы и Панчи
5993,49097575,0.156164,Молодёжный Балаково
7035,192271135,0.141883,Новости Балаково #KTA64
...,...,...,...
340986,188975915,0.000000,The Leafe HOSTEL
340987,189446501,0.000000,The Leafe HOSTEL
340988,189446540,0.000000,The Leafe Hostel
340989,668226217,0.000000,NaN


# Центральность по близости

*Были выбраны случайные 1000 узлов для оценки центральности, так как близость для каждого узла в графе определяется на основе расстояний до всех других узлов. Для каждого узла необходимо вычислить кратчайшие пути до всех остальных узлов графа, что только усугубляет время выполнения*

**Временная сложность алгоритма:** `O(V(V+E))`, **где V — количество узлов, а E — количество рёбер. То есть для каждого узла необходимо выполнить поиск кратчайших путей.**

In [21]:
import random

# Например, выбрать случайные 1000 узлов для оценки
subset_nodes = random.sample(list(G1.nodes()), 1000)
closeness_centrality_subset = {node: nx.closeness_centrality(G1, u=node) for node in subset_nodes}

# Сортируем по значению центральности
closeness_centrality_subset = sorted(closeness_centrality_subset.items(), key=lambda i: i[1], reverse=True)


In [22]:
closeness_centrality_subset[:10]

[(45745333, 0.2678734985759391),
 (31480508, 0.2555299351252035),
 (108468, 0.25010481635947684),
 (184856829, 0.2453374870273137),
 ('539481165', 0.24410277653829862),
 (33494375, 0.24338286130771064),
 ('557324677', 0.24281098910901136),
 ('114626038', 0.23912528259107535),
 ('141461696', 0.23848551680416752),
 (38316314, 0.23825681614763375)]

# Центральность по близости - результат
Группы с высокой центральностью по близости имеют высокие значения в столбце `weight`. Это означает, что группы занимают **центральное положение** в сети. Они соединяют разные части социальной сети, сокращая "расстояния" между участниками.

In [23]:
map_df(closeness_centrality_subset).drop_duplicates()

,group_id,weight,group_name
0,45745333,0.267873,4ch
1091,31480508,0.255530,Пикабу
1334,108468,0.250105,Кинопоиск
1450,184856829,0.245337,Trashgram
1579,539481165,0.244103,NaN
...,...,...,...
4630,190928314,0.130249,Wheel
4631,65041609,0.126751,КОНТЕКСТНАЯ РЕКЛАМА
4632,160590872,0.121043,Мастер группа ЕГЭ (БАЗА) - EXTRA
4633,219771017,0.000108,Подслушано Уварово (Тамбовская обл.)
